In [1]:
from bopt.core.lattice import Tokenizer
import math

In [16]:
vocab = ["[PAD]", "h","a","t","e", "at", "hat", "ate", "hate", "b", "ab"]

In [17]:
weight = {
    "[PAD]":0.0,
    "h":0.0,
    "a":0.0,
    "t":0.0,
    "e":math.log(1.0),
    "at": math.log(2.0), 
    "hat": math.log(3.0), 
    "ate": math.log(4.0), 
    "hate": math.log(5.0),
    "b": 0.0,
    "ab": 0.0,
}

In [18]:
tokenizer = Tokenizer(vocab, weight, continuing_subword_prefix=None)

In [19]:
ts, ms, la, ela, lb, elb, e, ee = tokenizer.forward(["hat", "hate",  "ab", "at"])

In [20]:
c, aa, bb, mm_, mm = tokenizer.conditionals(ts,ms,  la, ela, lb, elb)

In [21]:
for i,x in enumerate(["h", "ha", "hat", "hate", "a", "at", "ate", "t", "te", "e"]):
    for j, y in enumerate(["h", "ha", "hat", "hate", "a", "at", "ate", "t", "te", "e"]):
        print(f"[{x:>4} -> {y:>4}] {round(math.exp(c[1,i,j].item()),2)}", end=" ")
    print()

[   h ->    h] 1.0 [   h ->   ha] 0.0 [   h ->  hat] 0.0 [   h -> hate] 0.0 [   h ->    a] 0.14 [   h ->   at] 0.29 [   h ->  ate] 0.57 [   h ->    t] 0.14 [   h ->   te] 0.0 [   h ->    e] 0.43 
[  ha ->    h] 0.0 [  ha ->   ha] 0.0 [  ha ->  hat] 0.0 [  ha -> hate] 0.0 [  ha ->    a] 0.0 [  ha ->   at] 0.0 [  ha ->  ate] 0.0 [  ha ->    t] 0.0 [  ha ->   te] 0.0 [  ha ->    e] 0.0 
[ hat ->    h] 0.0 [ hat ->   ha] 0.0 [ hat ->  hat] 1.0 [ hat -> hate] 0.0 [ hat ->    a] 0.0 [ hat ->   at] 0.0 [ hat ->  ate] 0.0 [ hat ->    t] 0.0 [ hat ->   te] 0.0 [ hat ->    e] 1.0 
[hate ->    h] 0.0 [hate ->   ha] 0.0 [hate ->  hat] 0.0 [hate -> hate] 1.0 [hate ->    a] 0.0 [hate ->   at] 0.0 [hate ->  ate] 0.0 [hate ->    t] 0.0 [hate ->   te] 0.0 [hate ->    e] 0.0 
[   a ->    h] 1.0 [   a ->   ha] 0.0 [   a ->  hat] 0.0 [   a -> hate] 0.0 [   a ->    a] 1.0 [   a ->   at] 0.0 [   a ->  ate] 0.0 [   a ->    t] 1.0 [   a ->   te] 0.0 [   a ->    e] 1.0 
[  at ->    h] 1.0 [  at ->   ha] 0.0 [ 

In [22]:
tokenizer.zero_grad()
c.exp()[1][0][4].backward(retain_graph=True)
tokenizer.weights.weight.grad

tensor([[ 0.0000],
        [ 0.0000],
        [ 0.1224],
        [ 0.1224],
        [ 0.0816],
        [-0.0204],
        [ 0.0000],
        [-0.0204],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]])

In [23]:
e

tensor([1.0114, 1.4898, 0.6931, 0.6365], grad_fn=<SqueezeBackward1>)

In [10]:
s = sum(- x/15 * math.log(x/15) for x in [1,2,3,4,5])
print(s)

1.4897503188505912


In [28]:
tokenizer.zero_grad()
e[3].backward(retain_graph=True)
tokenizer.weights.weight.grad

tensor([[ 0.0000],
        [ 0.0000],
        [ 0.1540],
        [ 0.1540],
        [ 0.0000],
        [-0.0770],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000]])

In [34]:
-(1 + math.log(2/3))/9 * (1*3-1*2)  -(1 + math.log(1/3)) / 9 * (0*3 - 1 * 1)

-0.07701635339554949

In [33]:
-(1 + math.log(1/3))/9 * (1*3-1*1)-(1 + math.log(2/3))/9 * (0*3-1*2)

0.15403270679109898